<a href="https://colab.research.google.com/github/afirth03/IMLOAssessment/blob/main/IMLOExam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flowers 102 Dataset Classification

In [8]:
import pandas as pd
import numpy as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets